# CrewAI Multi-Agents Demo

# Installs

In [1]:
pip install openai crewai crewai-tools langchain_community

Note: you may need to restart the kernel to use updated packages.


# Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import configparser
import openai
import os

config = configparser.ConfigParser()
config.read('../pruebas_openai/config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from langchain.chat_models import ChatOpenAI
import requests
from IPython.display import Markdown
import pandas as pd

# Artículos

In [5]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data = data.iloc[[9]]

In [6]:
# Texto de ejemplo
texto_noticia = data['contenido'].values[0]
texto_noticia

'Dice Esther Vivas (Sabadell, 1975) que "la literatura de la maternidad parte a menudo de la propia experiencia". ¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante? El caso de esta periodista, que se había dedicado previamente al estudio del sistema agroalimentario y de los movimientos sociales, no es, dice, una excepción. A partir de su embarazo y de las primeras búsquedas de información sobre cómo se podría desarrollar el parto, comenzó a preguntarse por qué no le habían hablado de todo aquello. Por qué todo el mundo aceptaba esa violencia obstétrica. Mamá desobediente (editado por Capitán Swing en castellano y en catalán por Ara Llibres, bajo el título Mama desobedient) es el resultado de aquella primera "indignación", pero no se limita a ella. En el ensayo, que lleva por subtítulo Una mirada feminista a la maternidad, Vivas se ocupa primero de los mitos existentes en torno a ella, para centrarse luego en dos aspectos concretos de esta experiencia que gene

# Define Agents

In [7]:
llm = ChatOpenAI(model_name=OPENAI_MODEL,
                 api_key= OPENAI_API_KEY,
                 temperature=0.2)

/var/folders/dh/_ks73kfn57l67vxm7b5y16qw0000gn/T/ipykernel_24705/4042902887.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=OPENAI_MODEL,


In [8]:
def crear_agente(tipo, profesion, genero, politica):
    pronombre = "ella" if genero == "mujer" else "él"
    articulo = "una" if genero == "mujer" else "un"
    role = f"{tipo.capitalize()} ({profesion}, {genero}, {politica})"

    contexto = (
        f"{articulo.capitalize()} {profesion} con identidad de género {genero} "
        f"y orientación política {politica}. {pronombre.capitalize()} aporta "
        f"una perspectiva particular sobre el lenguaje y los sesgos debido a su trasfondo."
    )

    # ----------------------
    # 1. Agente Analista
    # ----------------------
    if tipo == "analista":
        goal = "Detectar estereotipos, sesgos de género o lenguaje sexista en textos periodísticos."
        backstory = contexto + " Su labor es identificar sesgos implícitos o explícitos en el texto."

    # ----------------------
    # 2. Agente Crítico
    # ----------------------
    elif tipo == "critico":
        goal = "Explicar el impacto de los sesgos detectados sobre la percepción del rol de la mujer en la sociedad."
        backstory = contexto + " Analiza cómo estos sesgos afectan la forma en que se perciben las mujeres en contextos profesionales y sociales."

    # ----------------------
    # 3. Agente Redactor
    # ----------------------
    elif tipo == "redactor":
        goal = "Reescribir el texto original usando lenguaje inclusivo y eliminando estereotipos."
        backstory = contexto + " Se especializa en transformar lenguaje sesgado en narrativas igualitarias."

    # ----------------------
    # 4. Agente Comparador
    # ----------------------
    elif tipo == "comparador":
        goal = "Comparar los análisis de otros agentes y reflexionar sobre cómo la identidad influye en la detección de sesgos."
        backstory = contexto + " Observa cómo las variables de identidad (profesión, género, política) afectan la interpretación del texto."
    else:
        raise ValueError(f"Tipo de agente no válido: {tipo}")

    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        verbose=True,
        llm=llm
    )

In [9]:
# ----------------------
# 5. Agente Estilizador
# ----------------------
def crear_estilizador():
    return Agent(
        role="Estilizador (asistente técnico neutral)",
        goal="Convertir los resultados anteriores en un informe estructurado en Markdown.",
        backstory="Asistente especializado en presentación clara de resultados. No realiza análisis, sino que organiza y mejora la visualización del contenido.",
        verbose=True,
        llm=llm
    )


## Crear agentes

In [10]:
# Define los parámetros para cada uno de los 5 agentes funcionales
profesion = "jueza" # "jueza", "abogada", "periodista", "investigadora", "profesora", "médica", "psicóloga", "trabajadora social"
genero = "mujer" # "mujer", "hombre"
politica = "progresista" # "progresista", "conservadora", "neutra"

# Define los perfiles de los agentes
perfiles = {
    "analista":    {"profesion": profesion, "genero": genero, "politica": politica},
    "critico":     {"profesion": profesion, "genero": genero, "politica": politica},
    "redactor":    {"profesion": profesion, "genero": genero, "politica": politica},
    "comparador":  {"profesion": profesion, "genero": genero, "politica": politica},
}

In [11]:
# Crear agentes
agentes = {
    tipo: crear_agente(tipo, **params)
    for tipo, params in perfiles.items() if tipo != "estilizador"
}
agentes["estilizador"] = crear_estilizador()


## Crear tareas

In [12]:
tareas = [
    Task(
    description=(
            f"""Analiza el siguiente texto completo con atención al contexto y la intención del autor.

    Tu objetivo es detectar frases que perpetúen estereotipos, lenguaje sexista o sesgos de género de forma explícita o implícita.  
    ⚠️ Ten en cuenta que algunas frases podrían parecer problemáticas en sí mismas, pero no lo son si el texto las está usando para **criticar** esos sesgos.  
    No marques como sesgada ninguna frase si forma parte de una **denuncia o crítica** a ese mismo lenguaje.

    1. Enumera las frases que verdaderamente refuercen o reproduzcan estereotipos o sesgos de género. Justifica cada una.
    2. En el texto original:
        - Usa `<span style="color: red; font-weight: bold;">... </span>` para marcar frases con sesgos.
        - Usa `<span style="color: blue;">... </span>` para frases que denuncian o critican estereotipos.

    Texto original:
    {texto_noticia}
    """
        ),
        expected_output=(
            "Frases problemáticas con justificación + texto original anotado con Markdown."
        ),
        agent=agentes["analista"]
    ),
    Task(
        description="Explica cómo los sesgos detectados afectan la percepción de la mujer en la sociedad.",
        expected_output="Análisis crítico del impacto social.",
        agent=agentes["critico"]
    ),
    Task(
        description="Reescribe el texto con lenguaje inclusivo y sin estereotipos.",
        expected_output="Nueva versión del texto con lenguaje igualitario.",
        agent=agentes["redactor"]
    ),
    Task(
        description="Compara los análisis anteriores y reflexiona sobre cómo influyen la profesión, género y política de cada agente.",
        expected_output="Reflexión comparativa desde una perspectiva neutral.",
        agent=agentes["comparador"]
    ),
    Task(
        description="Toma todos los análisis anteriores y preséntalos en un informe en formato Markdown con títulos, listas, separadores y claridad visual.",
        expected_output="Informe en Markdown limpio y bien estructurado.",
        agent=agentes["estilizador"]
    )
]

# Orchestrate Agents

In [13]:
crew = Crew(
    agents=list(agentes.values()),
    tasks=tareas,
    verbose=True
)

# Execute

In [14]:
resultado = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9fd40c95-d073-4974-9c84-f6c7d8ca9c0d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4933d26a-b148-4454-8a19-71080930986b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4933d26a-b148-4454-8a19-71080930986b
       Status: Executing Task...
    └── 🤖 Agent: Analista (jueza, mujer, progresista)
            Status: In Progress

# Agent: Analista (jueza, mujer, progresista)
## Task: Analiza el siguiente texto completo con atención al contexto y la intención del autor.

    Tu objetivo es detectar frases que perpetúen estereotipos, lenguaje sexista o sesgos de género de forma explícita o implícita.  
    ⚠️ Ten en cuenta que algunas frases podrían parecer problemáticas en sí mismas, pero no lo son si el texto las está usando para **criticar** esos sesgos.  
    No marques como sesgada ninguna frase si forma parte de una **denuncia o crítica** a ese mismo lenguaje.

    1. Enumera las frases que verdaderamente refuercen o reproduzcan estereotipos o sesgos de género. Justifica cada una.
    2. En el texto original:
        - Usa `<span style="color: red; font-weight: bold;">... </span>` para marcar frases con sesgos.
        - Usa `<span style="color: blue;">... </span>` para frases que denuncian o critican estereotipos.

    Texto original:
    Dice Esther Vivas (Sabadell, 1975) que "la literatura de la maternid

🚀 Crew: crew
└── 📋 Task: 4933d26a-b148-4454-8a19-71080930986b
       Status: Executing Task...
    └── 🤖 Agent: Analista (jueza, mujer, progresista)
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Estilizador (asistente técnico neutral)
## Final Answer: 
# Informe sobre la Maternidad y el Feminismo

## 1. Frases que refuerzan o reproducen estereotipos o sesgos de género

- **"¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante?"**  
  Justificación: Esta frase sugiere que la maternidad es un tema trivial y que solo se le puede dar importancia si se tiene una experiencia personal. Refuerza la idea de que el cuidado de los hijos es un asunto menor en comparación con otros temas.

- **"La maternidad ha sido utilizada por el patriarcado como un instrumento de control del cuerpo de las mujeres."**  
  Justificación: Aunque la frase critica el uso patriarcal de la maternidad, también perpetúa la idea de que la maternidad es inherentemente negativa y que las mujeres son víctimas pasivas de un sistema opresor, sin reconocer la agencia de las mujeres en su experiencia materna.

- **"Ante ese modelo de feminidad abnegada cuyo fin último es ser madre."**

🚀 Crew: crew
├── 📋 Task: 4933d26a-b148-4454-8a19-71080930986b
│      Assigned to: Analista (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista (jueza, mujer, progresista)
│           Status: ✅ Completed
├── 📋 Task: 108e91af-a8b0-42d7-a053-260c97c0254c
│      Assigned to: Critico (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Critico (jueza, mujer, progresista)
│           Status: ✅ Completed
├── 📋 Task: 95b072fa-624d-4826-a3c6-34037c69941a
│      Assigned to: Redactor (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Redactor (jueza, mujer, progresista)
│           Status: ✅ Completed
├── 📋 Task: c99a72fd-4c98-4329-97d3-1a8f6743caa6
│      Assigned to: Comparador (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Comparador (jueza, mujer, progresista)
│           Status: ✅ Completed
└── 📋 Task: 7bec3792-a154-448e-ac01-ea9e71d9cbcc
       Status: Executing Task...
    └── 🤖 Agent: Estilizador (asistente técnico neutral)
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4933d26a-b148-4454-8a19-71080930986b
│      Assigned to: Analista (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista (jueza, mujer, progresista)
│           Status: ✅ Completed
├── 📋 Task: 108e91af-a8b0-42d7-a053-260c97c0254c
│      Assigned to: Critico (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Critico (jueza, mujer, progresista)
│           Status: ✅ Completed
├── 📋 Task: 95b072fa-624d-4826-a3c6-34037c69941a
│      Assigned to: Redactor (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Redactor (jueza, mujer, progresista)
│           Status: ✅ Completed
├── 📋 Task: c99a72fd-4c98-4329-97d3-1a8f6743caa6
│      Assigned to: Comparador (jueza, mujer, progresista)
│      Status: ✅ Completed
│   └── 🤖 Agent: Comparador (jueza, mujer, progresista)
│           Status: ✅ Completed
└── 📋 Task: 7bec3792-a154-448e-ac01-ea9e71d9cbcc
       Assigned to: Estilizador (asistente técnico neutral)
       Status: ✅ Completed
    └── 🤖 Agent: Estilizador (asistente técnico neutral)
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7bec3792-a154-448e-ac01-ea9e71d9cbcc                                                                     │
│  Agent: Estilizador (asistente técnico neutral)                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9fd40c95-d073-4974-9c84-f6c7d8ca9c0d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
display(Markdown(resultado.raw))

# Informe sobre la Maternidad y el Feminismo

## 1. Frases que refuerzan o reproducen estereotipos o sesgos de género

- **"¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante?"**  
  Justificación: Esta frase sugiere que la maternidad es un tema trivial y que solo se le puede dar importancia si se tiene una experiencia personal. Refuerza la idea de que el cuidado de los hijos es un asunto menor en comparación con otros temas.

- **"La maternidad ha sido utilizada por el patriarcado como un instrumento de control del cuerpo de las mujeres."**  
  Justificación: Aunque la frase critica el uso patriarcal de la maternidad, también perpetúa la idea de que la maternidad es inherentemente negativa y que las mujeres son víctimas pasivas de un sistema opresor, sin reconocer la agencia de las mujeres en su experiencia materna.

- **"Ante ese modelo de feminidad abnegada cuyo fin último es ser madre."**  
  Justificación: Esta frase refuerza el estereotipo de que la maternidad es el único propósito de las mujeres, sugiriendo que ser madre es una obligación y no una elección personal.

- **"La lactancia materna permite la autonomía de la mujer, porque no necesita gastar dinero."**  
  Justificación: Esta afirmación puede implicar que las mujeres que no pueden amamantar o que eligen no hacerlo son menos autónomas, perpetuando la idea de que la maternidad y la crianza son la única vía hacia la libertad y la independencia.

- **"Dar el biberón per se no te hace más libre como mujer, ni es un mecanismo de igualdad."**  
  Justificación: Aunque la frase critica la idea de que el biberón es sinónimo de libertad, también puede implicar que las mujeres que optan por esta opción están eligiendo un camino menos feminista, lo que puede ser visto como un juicio sobre sus decisiones.

---

## 2. Texto Original Anotado

Esther Vivas (Sabadell, 1975) afirma que "la literatura de la maternidad parte a menudo de la propia experiencia".  
**¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante?**  
El caso de esta periodista, que se había dedicado previamente al estudio del sistema agroalimentario y de los movimientos sociales, no es, dice, una excepción. A partir de su embarazo y de las primeras búsquedas de información sobre cómo se podría desarrollar el parto, comenzó a preguntarse por qué no le habían hablado de todo aquello. Por qué todo el mundo aceptaba esa violencia obstétrica. *Mamá desobediente* (editado por Capitán Swing en castellano y en catalán por Ara Llibres, bajo el título *Mama desobedient*) es el resultado de aquella primera "indignación", pero no se limita a ella. En el ensayo, que lleva por subtítulo *Una mirada feminista a la maternidad*, Vivas se ocupa primero de los mitos existentes en torno a ella, para centrarse luego en dos aspectos concretos de esta experiencia que generan un debate encendido: primero, la atención al parto; después, la lactancia.

Y no es casualidad que cuestiones aparentemente privadas sobre si una mujer le da el pecho o no a su hijo generen polémica. La autora no solo defiende que la maternidad es un asunto público, imbricado en la economía y la política, sino que la sitúa en el centro del feminismo de clase y el anticapitalismo. Si no se la ha considerado así, defiende en el libro, es porque existen intereses para ello.

### Preguntas y Respuestas

**P. ¿Por qué ser madre y feminista supone ser desobediente?**  
**R.** Ser madre implica desobedecer porque vivimos en un sistema que es hostil a la maternidad y a la crianza, y si eres feminista y de repente eres madre, esa maternidad te reafirma en tus valores feministas.

**P. ¿Pensar la maternidad desde el feminismo supone repensar la idea que tenemos de la incorporación de la mujer al mundo laboral?**  
**R.** Sí. En general, desde un cierto feminismo, a la hora de plantear la igualdad en el mercado de trabajo, se ha señalado la maternidad, el cuidado y la crianza como una carga.

**P. ¿Cómo afecta esto a la manera en que el movimiento feminista se ha planteado la narrativa en torno a la igualdad en el trabajo?**  
**R.** Aquí también hemos visto distintos discursos y estrategias. Por un lado hay un cierto feminismo liberal que básicamente reivindica la igualdad para un sector de mujeres de clase media blanca.

**P. ¿Qué modelo, de los expuestos en el libro, considera más conveniente?**  
**R.** Se tendrían que ampliar ambos permisos con un periodo de tiempo muy superior al actual, como de hecho hacen muchos países nórdicos.

---

## 3. Reflexiones Finales

Los sesgos de género han permeado profundamente en la percepción social de la maternidad y el rol de la mujer, afectando no solo cómo se ve a las mujeres en contextos profesionales, sino también cómo se valoran sus experiencias y decisiones personales. Frases como "¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante?" subrayan un desprecio hacia la maternidad, sugiriendo que el cuidado de los hijos es trivial y que solo es relevante si se vive de manera personal.

Además, la afirmación de que "la maternidad ha sido utilizada por el patriarcado como un instrumento de control del cuerpo de las mujeres" perpetúa la noción de que las mujeres son meras víctimas de un sistema opresor. Este enfoque puede llevar a una visión negativa de la maternidad, donde se la considera una carga en lugar de una experiencia enriquecedora y empoderadora.

La violencia obstétrica, mencionada como "la última frontera de la violencia de género", refleja cómo las prácticas médicas y sociales deshumanizan a las mujeres en el proceso de parto. Esto no solo tiene consecuencias físicas y emocionales para las mujeres, sino que también perpetúa la idea de que su bienestar es secundario a la eficiencia del sistema médico.

Finalmente, es crucial que el feminismo contemporáneo reevalúe su relación con la maternidad, no como un obstáculo a la emancipación, sino como una experiencia que debe ser reivindicada y valorada. La maternidad debe ser vista como una cuestión política que requiere un cambio en el entorno socioeconómico, donde se reconozca su importancia y se apoye a las mujeres en sus decisiones. La lucha por la igualdad de género debe incluir la reivindicación de la maternidad como una experiencia digna y empoderadora, que no debe ser estigmatizada ni desvalorizada.

In [16]:
analista_output = resultado.tasks_output[0].raw
display(Markdown(analista_output))

1. Frases que refuerzan o reproducen estereotipos o sesgos de género:

   - **"¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante?"**  
     Justificación: Esta frase sugiere que la maternidad es un tema trivial y que solo se le puede dar importancia si se tiene una experiencia personal. Refuerza la idea de que el cuidado de los hijos es un asunto menor en comparación con otros temas.

   - **"la maternidad ha sido utilizada por el patriarcado como un instrumento de control del cuerpo de las mujeres"**  
     Justificación: Aunque la frase critica el uso patriarcal de la maternidad, también perpetúa la idea de que la maternidad es inherentemente negativa y que las mujeres son víctimas pasivas de un sistema opresor, sin reconocer la agencia de las mujeres en su experiencia materna.

   - **"ante ese modelo de feminidad abnegada cuyo fin último es ser madre"**  
     Justificación: Esta frase refuerza el estereotipo de que la maternidad es el único propósito de las mujeres, sugiriendo que ser madre es una obligación y no una elección personal.

   - **"la lactancia materna permite la autonomía de la mujer, porque no necesita gastar dinero"**  
     Justificación: Esta afirmación puede implicar que las mujeres que no pueden amamantar o que eligen no hacerlo son menos autónomas, perpetuando la idea de que la maternidad y la crianza son la única vía hacia la libertad y la independencia.

   - **"dar el biberón per se no te hace más libre como mujer, ni es un mecanismo de igualdad"**  
     Justificación: Aunque la frase critica la idea de que el biberón es sinónimo de libertad, también puede implicar que las mujeres que optan por esta opción están eligiendo un camino menos feminista, lo que puede ser visto como un juicio sobre sus decisiones.

2. Texto original anotado:

   Dice Esther Vivas (Sabadell, 1975) que "la literatura de la maternidad parte a menudo de la propia experiencia". <span style="color: red; font-weight: bold;">¿Quién iba, si no, a querer dedicar su tiempo a un asunto tan insignificante?</span> El caso de esta periodista, que se había dedicado previamente al estudio del sistema agroalimentario y de los movimientos sociales, no es, dice, una excepción. A partir de su embarazo y de las primeras búsquedas de información sobre cómo se podría desarrollar el parto, comenzó a preguntarse por qué no le habían hablado de todo aquello. Por qué todo el mundo aceptaba esa violencia obstétrica. Mamá desobediente (editado por Capitán Swing en castellano y en catalán por Ara Llibres, bajo el título Mama desobedient) es el resultado de aquella primera "indignación", pero no se limita a ella. En el ensayo, que lleva por subtítulo Una mirada feminista a la maternidad, Vivas se ocupa primero de los mitos existentes en torno a ella, para centrarse luego en dos aspectos concretos de esta experiencia que generan un debate encendido: primero, la atención al parto; después, la lactancia.

Y no es casualidad que cuestiones aparentemente privadas sobre si una mujer le da el pecho o no a su hijo generen polémica. La autora no solo defiende que la maternidad es un asunto público, imbricado en la economía y la política, sino que la sitúa en el centro del feminismo de clase y el anticapitalismo. Si no se la ha considerado así, defiende en el libro, es porque existen intereses para ello.

Pregunta. <span style="color: blue;">¿Por qué ser madre y feminista supone ser desobediente?</span> <span style="color: blue;">¿Desobediente con respecto al sistema neoliberal o con respecto a cierto feminismo?</span>

Respuesta. <span style="color: blue;">Ser madre implica desobedecer porque vivimos en un sistema que es hostil a la maternidad y a la crianza, y si eres feminista y de repente eres madre, esa maternidad te reafirma en tus valores feministas.</span> <span style="color: red; font-weight: bold;">¿Qué sucede? Que en general el feminismo ha tenido una relación compleja con la maternidad, porque en la medida en que la maternidad ha sido utilizada por el patriarcado como un instrumento de control del cuerpo de las mujeres,</span> <span style="color: blue;">cuando el feminismo se rebeló contra esta imposición, en los años sesenta o setenta, en la Segunda Ola, se creó un discurso antipaternal y antirreproductivo, y no se abordaron las contradicciones que implica la maternidad.</span> Por eso creo que desde el feminismo es necesario reivindicar la experiencia materna, no en clave romántica sino en clave de experiencia que tenemos que poder vivir como mujeres, al margen de las imposiciones y restricciones del sistema.

P. <span style="color: blue;">¿Pensar la maternidad desde el feminismo supone repensar la idea que tenemos de la incorporación de la mujer al mundo laboral?</span>

R. Sí. En general, desde un cierto feminismo, a la hora de plantear la igualdad en el mercado de trabajo, se ha señalado la maternidad, el cuidado y la crianza como una carga. Desde este punto de vista, ante la maternidad patriarcal impuesta, <span style="color: red; font-weight: bold;">ante ese modelo de feminidad abnegada cuyo fin último es ser madre,</span> se ha planteado que la alternativa era supeditar la crianza al mercado y al empleo. Es necesario cambiar la mirada, porque el problema no es la maternidad, sino el mercado, una sociedad que es hostil a unas prácticas que son fundamentales para la reproducción de la vida, como son gestar, parir, dar de mamar y cuidar.

P. <span style="color: blue;">¿Cómo afecta esto a la manera en que el movimiento feminista se ha planteado la narrativa en torno a la igualdad en el trabajo?</span>

R. Aquí también hemos vistos distintos discursos y estrategias. Por un lado hay un cierto feminismo liberal que básicamente reivindica la igualdad para un sector de mujeres de clase media blanca. En Estados Unidos vemos en el partido demócrata y que aquí vemos a medio camino entre el PSOE y Ciudadanos. Por eso creo que hay que reivindicar el feminismo del 99%, el que reclama la igualdad del conjunto de las mujeres y que señala que las desigualdades de género vienen también de la desigualdades de clase y de etnia, y que por eso es necesario acabar con las desigualdades socioeconómicas para conseguir una igualdad de género real.

P. <span style="color: blue;">Por lo heredado de la Segunda Ola, existe esta idea de que liberarse de la maternidad obligatoria ha sido algo emancipatorio, y que por tanto poner la maternidad en el centro del feminismo sería negar esa conquista. ¿Cómo se resuelve este conflicto?</span>

R. Lo que es necesario no es acabar con la experiencia de la maternidad, sino con lo que la activista feminista Adrienne Rich, ya en los años setenta, llamaba “la institución de la maternidad”. Es fundamental que la nueva oleada feminista incorpore la reivindicación de poder vivir la experiencia materna libre de imposiciones. La cuestión no es negar la maternidad sino reivindicarla. Esto no implica dar la razón a posturas reaccionarias en relación a la maternidad, sino reivindicar la maternidad en clave emancipadora. <span style="color: blue;">Yo como mujer tengo derecho a decidir cómo quiero vivir mi embarazo, mi parto y mi lactancia: esto es feminismo, como también es feminismo decidir si quiero tener un hijo o no, si quiero abortar o no.</span> Reivindicar una maternidad feminista va de la mano con reivindicar el pleno control sobre nuestro cuerpo como mujeres.

P. <span style="color: blue;">Le dedica parte del libro al debate sobre las bajas de maternidad y paternidad, y señala que las nuevas bajas intransferibles quieren reforzar la igualdad entre hombres y mujeres pero en realidad ignora la experiencia de la maternidad. ¿Qué posición cree que ha faltado en el debate?</span>

R. Se han obviado en gran medida las demandas de un sector del feminismo que reivindica una ampliación de los permisos de maternidad, porque en la actualidad el permiso de maternidad ni siquiera permite una lactancia materna en exclusiva hasta los seis meses, como recomiendan todas las instancias de salud. Y también es un debate que ha menospreciado el sentir de muchas madres. Hay varios estudios que señalan que cuando una mujer tiene un hijo, quiere estar con la criatura, y en cambio estos permisos iguales e intransferibles que se reivindican siguen estigmatizando la experiencia materna y la consideran una carga. <span style="color: blue;">Con esto no quiero decir que la maternidad sea tan solo una maternidad de las mujeres, es una responsabilidad de madres, de padres, de la sociedad en general, y que hay que darle el valor social, político y económico que realmente tiene.</span>

Por otro lado, es positivo plantear una ampliación del permiso de paternidad, pero hay que tener en cuenta que si este finalmente se amplía hasta las ocho semanas el año que viene, en tan solo tres años el permiso de paternidad habrá aumentado un 300%, pasando de una semana en 2015 a ocho semana en el 2019. <span style="color: red; font-weight: bold;">Cuando el permiso de maternidad en treinta años, desde el 89, no se ha movido una coma, y es extremadamente corto.</span> No se puede ampliar el permiso de paternidad sin reivindicar un aumento del de maternidad. Esto se dice que se hace en nombre de la igualdad, pero creo que no podemos atribuir la desigualdad en el mercado de trabajo solo a la experiencia materna, porque van mucho más allá.

P. <span style="color: blue;">¿En qué medida cree que estas bajas iguales e intransferibles contribuyen a estigmatizar la experiencia de la maternidad?</span>

R. Porque son unos permisos en los cuales básicamente se antepone el mercado y el empleo a la crianza. Porque si se considera, en relación al permiso de maternidad, que 16 semanas son suficientes para acompañar a la criatura después de nacer, son permisos que no tienen en cuenta esas necesidades de un bebé, que necesita estar con sus progenitores el máximo tiempo posible tras nacer, y que la fuente de alimento más beneficiosa es la leche materna.

Este es un elemento del que no se acostumbra a hablar, pero que es una medida profundamente clasista. De los permisos actuales solo se benefician los progenitores que trabajan o han trabajado y cotizan, por lo que los sectores con menores recursos económicos no tienen acceso a ningún tipo de ayuda por maternidad o paternidad. Y en segundo lugar estos permisos iguales e intransferibles hacen que si tú como madre quieres estar más tiempo con tu criatura, quieres darle de mamar hasta los seis meses, <span style="color: red; font-weight: bold;">solo aquellas mujeres con determinadas capacidades adquisitivas podrán hacerlo, porque solo ellas podrán tomarse una excedencia por su cuenta.</span> Es una idea clasista que convierte la crianza en un privilegio.

P. <span style="color: blue;">¿Qué modelo, de los expuestos en el libro, considera más conveniente?</span>

R. Se tendrían que ampliar ambos permisos con un periodo de tiempo muy superior al actual, como de hecho hacen muchos países nórdicos, que dan un peso central a lo que es la ayuda a la maternidad y la crianza. Porque aquí cada año con los informes de la caída de la natalidad nos escandalizamos, pero no se hace absolutamente nada para revertirla. Apostaría por un modelo como los nórdicos, donde las bajas son más amplias que las que tenemos en el Estado español, y que pueden ser compartidas y una parte de ellas transferibles.

P. <span style="color: blue;">¿Por qué la lactancia tiene tanto espacio en el debate sobre la maternidad? Se discute desde sobre cómo y dónde se produce hasta qué forma es más conveniente.</span>

R. El parto y la lactancia son dos partes esenciales en la crianza. Quería profundizar en el debate sobre la lactancia materna porque también en los feminismos hay un cierto debate sobre si se impone la teta o el biberón. Y me parecía interesante señalar e investigar cómo, a pesar de que cada vez se ha recuperado la lactancia materna —algo que es positivo, porque en los años setenta prácticamente desapareció—, aún el contexto social es adverso, contrariamente a lo que puede parecer en un primer momento.

P. <span style="color: blue;">En esa época se configura un discurso que asegura que el biberón da más libertad a las madres, y que además también facilita la incorporación al trabajo. ¿Cómo nos relacionamos hoy con estas dos ideas?</span>

R. Cuando yo reflexiono con una mirada crítica acerca de la lactancia artificial lo hago desde el punto de vista de los intereses económicos que hay detrás el negocio del biberón, no sobre lo que hace cada madre, porque por desgracia hacemos lo que buenamente podemos en nuestras circunstancias. Dicho esto, cuando se generalizó la lactancia artificial en los años setenta u ochenta, hay unos intereses empresariales claros. La generalización de la lactancia artificial supuso un negocio ingente para las empresas de la industria láctea y alimentaria, como Nestlé o Danone, de ahí que se promocionara como se hizo. Y lo que se hizo fue incidir en el personal sanitario para que este fuera proclive a promocionar la lactancia artificial en detrimento de la lactancia materna.

Por otro lado, uno de los elementos que ha incomodado más al feminismo con respecto a la lactancia es el carácter biológico de la práctica, que es biosocial, hay un elemento cultural y otro biológico. Yo creo que desde el feminismo se trata de reconocer tanto el carácter biológico como cultural de la maternidad y de la lactancia, pero precisamente por esto, fruto de esta incomodidad, muchas veces se ha considerado que la maternidad era una práctica que encerraba a las madres en el hogar, que las alejaba del mercado de trabajo, y es necesario cambiar la mirada. Porque dar de mamar es una práctica profundamente feminista, e incluso diría que antineoliberal y anticapitalista. En la medida en que dar la teta no es solo lo mejor para la criatura (y hay determinadas circunstancias en las que dar el biberón es lo mejor), también es extremadamente beneficiosa para la mamá y en particular después del parto, por lo que implica de recuperación, de vínculo con la criatura. A la vez, la lactancia materna permite la autonomía de la mujer, porque no necesita gastar dinero, sabe lo que le da de comer, es kilómetro cero… Y si decía que dar la teta es antineoliberal es porque es una práctica que no beneficia al mercado, que necesita de unos tiempos y de un espacio que son contrarios a la lógica mercantil y productivista, y no tiene un impacto negativo en el medio ambiente.

P. <span style="color: blue;">Las madres que optan por dar el pecho también suelen hablar de que esto supone una desigualdad insalvable con el padre en la tarea de cuidados.</span>

R. <span style="color: red; font-weight: bold;">Dar el biberón per se no te hace más libre como mujer, ni es un mecanismo de igualdad.</span> Basta con ver a las madres que dan el biberón y crían en soledad. Tampoco la teta per se te hace más libre. El carácter biológico de la lactancia es innegable, pero esto no implica que el padre no pueda implicarse en muchas otras tareas. Los bebés no solo se despiertan por la noche porque quieren mamar, sino por muchas otras circunstancias. Criar en igualdad no tiene tanto que ver con cómo alimentamos a la criatura nada más nacer, sino con muchos otros elementos vinculados a la relación de pareja y con un entorno social que es hostil a la maternidad y a la crianza.

P. <span style="color: blue;">La tercera pata del libro aborda la violencia obstétrica, y habla de una evolución progresiva en los hospitales. ¿Cuál es el panorama general ahora mismo?</span>

R. Ha habido una serie de cambios muy significativos en relación a la atención al parto en el sistema hospitalario, que se han producido gracias y en parte a organizaciones de mujeres que han reivindicado un parto respetado. Sin <span style="color: blue;">El parto es nuestro</span> no se entenderían los cambios en la atención sanitaria. Pero aún así, en la medida en que muchas de estas modificaciones que se han hecho a nivel gubernamental son de cumplimiento voluntario, aún existen muchas malas prácticas del personal sanitario, debido a que hay una serie de mitos y prejuicios en relación al nacimiento que acaban justificando estas prácticas. Y de hecho solo es necesario ver los porcentajes de cesáreas, de episiotomías, de partos instrumentales que se llevan a cabo, que son muy superiores de los que la Organización Mundial de la Salud considera razonables. Esto tiene un impacto físico y psíquico en las mujeres, y también en las criaturas.

P. <span style="color: blue;">Llama particularmente la atención las cifras de episiotomía, que indican que se dan en España en el 43% de los partos frente al tope del 15% que recomienda la OMS. ¿Hay una carga cultural en esto?</span>

R. Más que una carga cultural hay unas prácticas médicas que consideran que en la medida en que el parto es una enfermedad, la atención al parto se aborda desde este punto de vista y esto dificulta que el proceso del parto se dé de manera natural. Estas prácticas facilitan el trabajo médico, pero tienen consecuencias negativas. En el caso de la episiotomía, particularmente, por lo que implica de problemas de micción, en las relaciones sexuales, etcétera. Hoy, cuatro de cada 10 mujeres que dan a luz sufren una episiotomía: hemos mejorado, porque en el año 95 eran 9 de cada 10. En los países nórdicos es 1 de cada 10. No creo que el periné de las mujeres españolas sea distinto al periné de las mujeres suecas. Si esto se produce es porque hay detrás unas prácticas médicas que consideran que esto es correcto. Hay que cambiar la formación médica para darle una perspectiva feminista y de género.

P. <span style="color: blue;">En el libro se pregunta dónde están las comadronas, y también se plantea el problema de la formación. ¿Cómo se relacionan estos dos aspectos, qué se aprende en las universidades y quienes llegan a atender el parto?</span>

R. Decíamos que hay una visión por parte del personal sanitario patológica, que se ve como una enfermedad, y esta visión es compartida por la sociedad. Y en la medida en que la sociedad no reconoce el parto como un proceso fisiológico, es difícil conseguir que haya un parto respetado. <span style="color: red; font-weight: bold;">Yo digo que la violencia obstétrica es la última frontera de la violencia de género porque es una violencia aceptada, por los profesionales, por las mujeres afectadas y por sus allegados.</span> Se considera que el parto es eso. Y por eso pensamos siempre que el personal mejor preparado para atender al parto es un ginecólogo, cuando en los partos de bajo riesgo es la comadrona. Hay que reivindicar la figura de la comadrona en la atención al parto y su plena autonomía como profesionales de la salud.

P. <span style="color: blue;">¿comadronas y madres son aliadas?</span>

R. Sí, por supuesto. Lo que señalan muchos estudios es que en la medida en que el parto es llevada por comadronas, el grado de satisfacción de las mujeres que dan a luz es superior. El problema es que hay una profunda relación jerárquica en el sistema hospitalario que tienes a menospreciarlas. A la vez hay una falta también de este personal en los hospitales, que va en detrimento de la calidad de la atención al parto. Que haya más comadronas revierte positivamente en la salud de las mujeres.

El 85% de excedencias para el cuidado de hijos tramitadas hasta marzo de 2021 fueron solicitadas por las madres

P. <span style="color: blue;">¿Cuál considera que debe ser el lugar desde el que se reclamen estas reivindicaciones alrededor de la maternidad?</span>

R. Por un lado, es necesario que el feminismo reivindique la maternidad y la experiencia materna al margen de las imposiciones del sistema. Creo que esta nueva ola feminista, este feminismo del 99%, incorpore la maternidad a sus demandas y que se normalice la relación entre feminismos y maternidad. No como algo idílico o romántico, sino como experiencia y dándole el valor sociopolítico y económico que tiene. Luego están los grupos de crianza, los grupos de apoyo a la lactancia materna, creo que es importante que se sumen a esta oleada feminista en auge y que apuesten por una maternidad transformadora a la hora de llevar a cabo sus reivindicaciones. Porque la maternidad no es una cuestión individual, es una cuestión política. Porque si queremos vivir una maternidad digna lo que tenemos que hacer es cambiar el entorno socioeconómico. Y este cambio radical del sistema solo puede hacerse con el apoyo de otros movimientos sociales.